In [ ]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf

pd.set_option('display.max_columns', None)

%reload_kedro

In [ ]:
df = io.load("clean_automobile")

In [ ]:
df.head()

In [ ]:
df.groupby(["make", "num_cylinders"])["price"].mean().dropna().sort_values()

In [ ]:
mod = smf.ols("price ~ make:num_cylinders - 1", df)
res = mod.fit()
res.summary2().tables[1].dropna().round(1).sort_values("Coef.")